In [ ]:
# standard lib
from datetime import datetime, date
from os import environ

# 3rd party
import pandas as pd
from pytz import timezone
import matplotlib as plt

# repo specific
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\finx_ib_reports")

from finx_ib_reports.download_trades import load_report
from finx_ib_reports.custom_flex_report import CustomFlexReport

In [ ]:
# pull in dev env vars
import dotenv
from os import environ

# this is a single IBKR account, eg, U123456789
# injecting as an env var to keep it private
configs = dotenv.dotenv_values('../.env.dev')
ACCOUNT_ID = configs.get('ACCOUNT_ID')

In [ ]:
fn = "../data/flex_report_1651436239.xml"
report = load_report(fn)

In [ ]:
# we want to calculate returns for both closed positions and open positions

# open positions df
opdf = report.open_positions_by_account_id(account_id=ACCOUNT_ID)

# closed trades df
tdf = report.trades_by_account_id(account_id=ACCOUNT_ID)

In [ ]:
def set_finx_instrument_symbol(df: pd.DataFrame) -> pd.DataFrame:
    """
    For STK, OPT, FUT, and FOP, set the 'finx_instrument_symbol'.
    This method mutates the df.
    Returns the mutated df for convenience.
    """
    assert "description" in df.columns, "missing 'description' in df"
    assert "assetCategory" in df.columns, "missing 'assetCategory' in df"

    default = df[ df["assetCategory"].isin(["FUT", "OPT"]) ]
    df.loc[default.index, "finx_instrument_symbol"] = default["underlyingSymbol"]
    
    fop = df[ df["assetCategory"] == "FOP" ]
    df.loc[fop.index, "finx_instrument_symbol"] = fop["description"].str.split("\s", expand=True).iloc[:, 0]

    stk = df[ df["assetCategory"] == "STK" ]
    df.loc[stk.index, "finx_instrument_symbol"] = stk["symbol"]
    
    return df


def get_latest_report_date_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns copy of df containing only most recent rows
    """
    most_recent_reportDate = max(df["reportDate"])
    return df[df["reportDate"] == most_recent_reportDate].copy()


def concat_positions_and_closed_trades(positions_df: pd.DataFrame, closed_trades_df: pd.DataFrame) -> pd.DataFrame:
    """
    Concat the open position df and the closed trades df
    """
    closed_trades_df["finx_dt"] = pd.to_datetime(closed_trades_df["dateTime"])
    et = timezone('US/Eastern')
    positions_df["finx_dt"] = pd.to_datetime(positions_df["reportDate"]).dt.tz_localize(et)
    df = pd.concat([
        closed_trades_df[["finx_dt", "finx_instrument_symbol", "fifoPnlRealized"]], 
        positions_df[["finx_dt", "finx_instrument_symbol", "fifoPnlUnrealized"]]
    ])
    return df

def gen_mapper_dict():
    # this might change based on markets/strategies traded
    tag_def = {
        "Oil": ["CL", "MCL"],
        "EquityIndex": ["ES", "MES", "NQ", "MNQ", "RTY", "M2K"],
        "Corn": ["ZC", "YC"],
        "Rates": ["ZN"],
        "Gold": ["MGC", "GC"],
    }
    mapper_dict = {}
    for k,v in tag_def.items():
        for product in v:
            mapper_dict[product] = k
    return mapper_dict



def categorize(row: tuple) -> str:
    """
    Set 'category' based on 'finx_instrument_symbol'
    """
    mapper_dict = gen_mapper_dict()
    # if key exists in dictionary
    if row['finx_instrument_symbol'] in mapper_dict:
        # return the key's value
        return mapper_dict[row['finx_instrument_symbol']]
    else:
        # if not, mark as
        return "UnCategorized"
    

In [ ]:
# set report start date and end date
et = timezone('US/Eastern')
sd, ed = datetime(2022, 1, 1, tzinfo=et), datetime(2022, 5, 1, tzinfo=et)

# prepare the trade df
tdf = set_finx_instrument_symbol(tdf)
tdfx = tdf.query("@sd < dateTime and dateTime < @ed").copy()

# prepare the open position df
opdf = set_finx_instrument_symbol(opdf)
opdf = opdf.query("@sd.date() < reportDate and reportDate < @ed.date()")
opdfx = get_latest_report_date_rows(opdf)

# # merge open positions and trades
df = concat_positions_and_closed_trades(opdfx, tdfx)

In [ ]:
#
# End goal, group all instruments into the categories we trade and plot. 
# categories: Crude, EquityVol, Rates, Corn, Gold, and UnCategorized
#
# underlying df
udf = df.groupby(["finx_instrument_symbol"]).agg({"fifoPnlRealized": "sum", "fifoPnlUnrealized": "sum"}).reset_index()
udf['category'] = udf.apply(lambda row: categorize(row), axis=1)
udf["totalPnL"] = udf["fifoPnlUnrealized"] + udf["fifoPnlRealized"]

# category df
cdf = udf.groupby(["category"]).agg({"totalPnL": "sum", "fifoPnlRealized": "sum"}).reset_index().set_index(["category"])
cdf["Percent"] = (cdf["totalPnL"] / cdf['totalPnL'].sum()) * 100.0

# plot cdf
axes = cdf[["totalPnL", "Percent", "fifoPnlRealized"]].plot.bar(rot=0, subplots=True, figsize=(10, 14))
for ax in axes:
    ax.legend(loc=2)

# format dollar values
axes[0].get_yaxis().set_major_formatter(plt.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axes[2].get_yaxis().set_major_formatter(plt.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

# format percent values
axes[1].get_yaxis().set_major_formatter(plt.ticker.PercentFormatter())